# 🚚 Distributed Dataframes

In [1]:
import webbrowser

import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client as DaskClient
from sklearn import datasets

In [2]:
# load boston dataset from sklearn
boston = datasets.load_boston()

# load boston dataset as pandas dataframe
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [3]:
# create Dask scheduler, checking whether it already exists or not from the local runtime for this notebook
if "daskclient" not in locals():
    daskclient = DaskClient()
print("Dask Scheduler Address: " + str(daskclient.scheduler_info()["address"]))

Dask Scheduler Address: tcp://127.0.0.1:60685


In [4]:
# open a webbrowser to view dask status dynamically
webbrowser.open("http://127.0.0.1:8787/status")

True

In [5]:
# load the pandas df into dask dataframe
dask_df = dd.from_pandas(data=df, npartitions=3)
dask_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [6]:
dask_df = dask_df.apply(
    lambda x: (((x ** 10) / 0.345) ** 0.55) / 0.01,
    axis=1,
    meta={
        "CRIM": "float64",
        "ZN": "float64",
        "INDUS": "float64",
        "CHAS": "float64",
        "NOX": "float64",
        "RM": "float64",
        "AGE": "float64",
        "DIS": "float64",
        "RAD": "float64",
        "TAX": "float64",
        "PTRATIO": "float64",
        "B": "float64",
        "LSTAT": "float64",
    },
)
dask_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.439274e-10,1.439456e+09,1.795004e+04,0.0,5.936111,5.657521e+06,1.708289e+12,4.156021e+05,179.555825,7.019463e+15,5.888469e+08,3.523267e+16,1.227329e+06
1,4.507844e-07,0.000000e+00,8.433469e+06,0.0,2.790295,4.966085e+06,4.876666e+12,1.209945e+06,8125.769035,2.318377e+15,1.353660e+09,3.523267e+16,3.462610e+07
2,4.489717e-07,0.000000e+00,8.433469e+06,0.0,2.790295,9.216091e+06,1.195160e+12,1.209945e+06,8125.769035,2.318377e+15,1.353660e+09,3.329086e+16,3.831574e+05
3,1.148112e-06,0.000000e+00,1.305300e+04,0.0,2.448836,7.971796e+06,2.448836e+11,3.619645e+06,75572.954068,1.442583e+15,1.775531e+09,3.413855e+16,6.762540e+04
4,7.406274e-05,0.000000e+00,1.305300e+04,0.0,2.448836,8.951180e+06,6.182948e+11,3.619645e+06,75572.954068,1.442583e+15,1.775531e+09,3.523267e+16,1.783196e+06
